## Genrate Train List

In [ ]:
import os,json,csv

trainRoot = "/home/edward/test/OUTtest/CMU_AICenter/NIH_CXR_Images/train/"
testRoot = "/home/edward/test/OUTtest/CMU_AICenter/NIH_CXR_Images/test/"
trainList='/home/edward/test/OUTtest/CMU_AICenter/trainReal1.json'
fileDict ={
    'train' : [],
    'test' : []
    }

trainCsv = '/home/edward/test/OUTtest/CMU_AICenter/NIH_CXR_train.csv'
testCsv = '/home/edward/test/OUTtest/CMU_AICenter/NIH_CXR_test.csv'


trainLabelDict = {}
testLabelDict = {}

with open(trainCsv) as f:
    rows = csv.reader(f)
    for row in rows:
#         print(row)
        if('png' in row[0]):
            trainLabelDict[row[0]]=row[1]
# print(trainLabelDict)

with open(testCsv) as f:
    rows = csv.reader(f)
    for row in rows:
#         print(row)
        if('png' in row[0]):
            testLabelDict[row[0]]=row[1]
# print(testLabelDict)


for fp,dl,fl in os.walk(trainRoot):
    # print(fp,dl,fl)
    if(len(fl)>0):
        for i in fl:
            if(i.split('.')[-1]=='png'):
                # print(fp + i)
                if(trainLabelDict[i] == 'Atelectasis'):
                    label = 1
                if(trainLabelDict[i] == 'No Finding'):
                    label = 0
                # print(fp + '/' + i,label)
                # with open(trainList,'a') as f:
                #     f.writelines(fp + '/' + i + ',' + str(label) + '\n')
                fileDict['train'].append([fp + i,label])

for fp,dl,fl in os.walk(testRoot):
    # print(fp,dl,fl)
    if(len(fl)>0):
        for i in fl:
            if(i.split('.')[-1]=='png'):
                # print(fp + i)
                if(testLabelDict[i] == 'Atelectasis'):
                    label = 1
                if(testLabelDict[i] == 'No Finding'):
                    label = 0
                # print(fp + '/' + i,label)
                # with open(trainList,'a') as f:
                #     f.writelines(fp + '/' + i + ',' + str(label) + '\n')
                fileDict['test'].append([fp + i,label])

with open(trainList,'w') as f:
    json.dump(fileDict,f,indent=4)


## Train

In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras.utils import to_categorical
from tensorflow.keras.layers import Input
from keras import backend as K
import os, datetime, tensorflow as tf, numpy as np,cv2,json


from skimage.io import imread
from skimage.transform import resize
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())
# print(tf.__version__)
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.test.is_gpu_available()

def binary_focal_loss(gamma=2, alpha=0.25):
    alpha = tf.constant(alpha, dtype=tf.float32)
    gamma = tf.constant(gamma, dtype=tf.float32)

    def binary_focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        alpha_t = y_true*alpha + (K.ones_like(y_true)-y_true)*(1-alpha)
    
        p_t = y_true*y_pred + (K.ones_like(y_true)-y_true)*(K.ones_like(y_true)-y_pred) + K.epsilon()
        focal_loss = - alpha_t * K.pow((K.ones_like(y_true)-p_t),gamma) * K.log(p_t)
        return K.mean(focal_loss)
    return binary_focal_loss_fixed

class datagenator(tf.keras.utils.Sequence):

    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=10, shuffle=True,n_classes = 2,mode='train'):
        'Initialization'
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.n_classes = n_classes
        self.mode = mode
        self.err={
            'effective':[],
            'invalid':[]
        }
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        # print(80*'!')
        # print(np.shape(X),np.shape(y))

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = [] #np.empty((self.batch_size, *self.dim, self.n_channels))
        y = [] #np.empty((self.batch_size), dtype=int)

        # Generate data
        for ID in list_IDs_temp:
            # Store sample
            img = cv2.imread(ID)
            in_ = np.array(img, dtype=np.float32)
            shape=np.shape(in_)
            try:
                side = 517
                if(shape[0]>shape[1]):
                        in_=cv2.resize(in_,(side,shape[1]*side/shape[0]),interpolation=cv2.INTER_CUBIC)
                        newshape=np.shape(in_)
                        addsp=(side-newshape[0])/2
                        fr_sp=np.zeros((addsp,side,3),np.float32)
                        if((side-newshape[0])%2!=0):
                                addsp+=1
                        bk_sp=np.zeros((addsp,side,3),np.float32)
                        in_=np.concatenate((fr_sp,in_,bk_sp),axis=0)
                elif(shape[0]<shape[1]):
                        in_=cv2.resize(in_,(shape[0]*side/shape[1],side),interpolation=cv2.INTER_CUBIC)
                        newshape=np.shape(in_)
                        addsp=(side-newshape[1])/2
                        fr_sp=np.zeros((side,addsp,3),np.float32)
                        if((side-newshape[1])%2!=0):
                                addsp+=1
                        bk_sp=np.zeros((side,addsp,3),np.float32)
                        in_=np.concatenate((fr_sp,in_,bk_sp),axis=1)

                else:
                        in_=cv2.resize(in_,(side,side),interpolation=cv2.INTER_CUBIC)
            except:
                in_ = np.array(np.random.randint(0,256,size = (517,517,3)), dtype=np.float32)
                # print(ID)
                if('effective' in ID):
                    if not(ID.split('effective')[-1].split('/')[1] in self.err['effective']):
                        self.err['effective'].append(ID.split('effective')[-1].split('/')[1])
                        self.err['effective'].sort()
                if('invalid' in ID):
                    if not(ID.split('invalid')[-1].split('/')[1] in self.err['invalid']):
                        self.err['invalid'].append(ID.split('invalid')[-1].split('/')[1])
                        self.err['invalid'].sort()

                if(self.mode =='train'):
                    with open('err_train.json','w') as f:
                        # f.writelines(ID + '\n')
                        json.dump(self.err,f,indent=4)
                
                if(self.mode =='val'):
                    with open('err_test.json','w') as f:
                        # f.writelines(ID + '\n')
                        json.dump(self.err,f,indent=4)
            
            X.append(in_)

            # Store class
            y.append(self.labels[ID])

        return np.array(X), np.array(to_categorical(y, num_classes=self.n_classes))


imgList = json.load(open('/home/edward/test/OUTtest/CMU_AICenter/trainReal1.json'))

partition = {
    'train' :[],
    'test' :[]
}
labels = {}

for i in imgList['train']:
    partition['train'].append(i[0])
    labels[i[0]] = i[1]
for i in imgList['test']:
    partition['test'].append(i[0])
    labels[i[0]] = i[1]

# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(517, 517, 3))
# create the base pre-trained model
base_model = InceptionV3(   
    input_tensor=input_tensor,
    weights=None,
    include_top=True,
    classes=2,
) #input_tensor=input_tensor, weights='imagenet', 

# add a global spatial average pooling layer
x = base_model.output
# x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
# x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
# predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=x)
# model.summary()

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy', 'Precision', 'Recall', 'AUC']) 
#'categorical_crossentropy' [binary_focal_loss(alpha=.25, gamma=2)]

# model.load_weights("/home/cwlab/edward/keras_test/CODE/checkpoint/bk/weights.11-1.00.hdf5", by_name=True)

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
checkPointPath = './checkpoint/weights.{epoch:02d}-{auc:.2f}.hdf5'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkPointPath, monitor='auc', save_weights_only=True)

training_generator = datagenator(partition['train'], labels, batch_size=10, n_classes = 2, shuffle=True, mode='train')
val__generator = datagenator(partition['test'], labels, batch_size=10, n_classes = 2, shuffle=True,mode='val')
# x_train = np.random.randint(0,256,size=(1000,517,517,3))
# print(np.shape(x_train))
# y_train=to_categorical(np.random.randint(0,2,size=(1000)))
# print('y_train= ',y_train)
# print([[0.,1.]])

model.fit(
    x = training_generator, 
    epochs = 30,
    callbacks = [tensorboard_callback, checkpoint_callback],
    validation_data = val__generator
    ) #validation_data=(x_test, y_test),batch_size = 10, 

log_writer = tf.summary.create_file_writer(logdir)

model.save_weights(os.path.join("weights", datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '.h5'))

Epoch 1/30
  1/424 [..............................] - ETA: 0s - loss: 0.0124 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000WARNING:tensorflow:From /home/edward/.local/lib/python3.6/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
 25/424 [>.............................] - ETA: 1:37 - loss: 0.1205 - accuracy: 0.9080 - precision: 0.9080 - recall: 0.9080 - auc: 0.9228

KeyboardInterrupt: 

## Test

In [1]:
from keras.utils import to_categorical
# from tensorflowf.keras.utils import Sequence
from tensorflow.keras.layers import Input
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from keras import backend as K
import numpy as np,cv2,tensorflow as tf,json,os,datetime

# y=np.random.randint(0,4,size=(1,512,512,3))
# y_test=to_categorical(y)
# print(y_test)

def binary_focal_loss(gamma=2, alpha=0.25):
    alpha = tf.constant(alpha, dtype=tf.float32)
    gamma = tf.constant(gamma, dtype=tf.float32)

    def binary_focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        alpha_t = y_true*alpha + (K.ones_like(y_true)-y_true)*(1-alpha)
    
        p_t = y_true*y_pred + (K.ones_like(y_true)-y_true)*(K.ones_like(y_true)-y_pred) + K.epsilon()
        focal_loss = - alpha_t * K.pow((K.ones_like(y_true)-p_t),gamma) * K.log(p_t)
        return K.mean(focal_loss)
    return binary_focal_loss_fixed

class datagenator(tf.keras.utils.Sequence):

    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=10, shuffle=True,n_classes = 2,mode='train'):
        'Initialization'
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.n_classes = n_classes
        self.mode = mode
        self.err={
            'effective':[],
            'invalid':[]
        }
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        # print(80*'!')
        # print(np.shape(X),np.shape(y))

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = [] #np.empty((self.batch_size, *self.dim, self.n_channels))
        y = [] #np.empty((self.batch_size), dtype=int)

        # Generate data
        for ID in list_IDs_temp:
            # Store sample
            img = cv2.imread(ID)
            in_ = np.array(img, dtype=np.float32)
            shape=np.shape(in_)
            try:
                side = 517
                if(shape[0]>shape[1]):
                        in_=cv2.resize(in_,(side,shape[1]*side/shape[0]),interpolation=cv2.INTER_CUBIC)
                        newshape=np.shape(in_)
                        addsp=(side-newshape[0])/2
                        fr_sp=np.zeros((addsp,side,3),np.float32)
                        if((side-newshape[0])%2!=0):
                                addsp+=1
                        bk_sp=np.zeros((addsp,side,3),np.float32)
                        in_=np.concatenate((fr_sp,in_,bk_sp),axis=0)
                elif(shape[0]<shape[1]):
                        in_=cv2.resize(in_,(shape[0]*side/shape[1],side),interpolation=cv2.INTER_CUBIC)
                        newshape=np.shape(in_)
                        addsp=(side-newshape[1])/2
                        fr_sp=np.zeros((side,addsp,3),np.float32)
                        if((side-newshape[1])%2!=0):
                                addsp+=1
                        bk_sp=np.zeros((side,addsp,3),np.float32)
                        in_=np.concatenate((fr_sp,in_,bk_sp),axis=1)

                else:
                        in_=cv2.resize(in_,(side,side),interpolation=cv2.INTER_CUBIC)
            except:
                in_ = np.array(np.random.randint(0,256,size = (517,517,3)), dtype=np.float32)
                # print(ID)
                if('effective' in ID):
                    if not(ID.split('effective')[-1].split('/')[1] in self.err['effective']):
                        self.err['effective'].append(ID.split('effective')[-1].split('/')[1])
                        self.err['effective'].sort()
                if('invalid' in ID):
                    if not(ID.split('invalid')[-1].split('/')[1] in self.err['invalid']):
                        self.err['invalid'].append(ID.split('invalid')[-1].split('/')[1])
                        self.err['invalid'].sort()

                if(self.mode =='train'):
                    with open('err_train.json','w') as f:
                        # f.writelines(ID + '\n')
                        json.dump(self.err,f,indent=4)
                
                if(self.mode =='val'):
                    with open('err_test.json','w') as f:
                        # f.writelines(ID + '\n')
                        json.dump(self.err,f,indent=4)
            
            X.append(in_)

            # Store class
            y.append(self.labels[ID])

        return np.array(X), np.array(to_categorical(y, num_classes=self.n_classes))

imgList = json.load(open('/home/edward/test/OUTtest/CMU_AICenter/trainReal1.json'))

partition = {
    'test' :[]
}
labels = {}

for i in imgList['test']:
    partition['test'].append(i[0])
    labels[i[0]] = i[1]

val__generator = datagenator(partition['test'], labels, batch_size=10, n_classes = 2, shuffle=True, mode='val')

input_tensor = Input(shape=(517, 517, 3))
base_model = InceptionV3(   
    input_tensor=input_tensor,
    weights=None,
    include_top=True,
    classes=2,
)
x = base_model.output

model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer='adam', loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=['accuracy', 'Precision', 'Recall', 'AUC'])#'categorical_crossentropy' [binary_focal_loss(alpha=.25, gamma=2)] 

model.load_weights("/home/edward/test/OUTtest/CMU_AICenter/Focal Loss/weights.22-0.96.hdf5", by_name=True)

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

#==============evaluate======================
evalValue = model.evaluate(
    x=val__generator,
    callbacks=tensorboard_callback,
    return_dict=True,
)
print(evalValue)


print('...')

29/29 [==============================] - 4s 125ms/step - loss: 0.0699 - accuracy: 0.9069 - precision: 0.9069 - recall: 0.9069 - auc: 0.9104
{'loss': 0.06994660943746567, 'accuracy': 0.9068965315818787, 'precision': 0.9068965315818787, 'recall': 0.9068965315818787, 'auc': 0.9103567004203796}
...


### RMSprop + categorical_crossentropy

{'loss': 0.43966642022132874, 'accuracy': 0.9068965315818787, 'precision': 0.9068965315818787, 'recall': 0.9068965315818787, 'auc': 0.9028180241584778}

### ADAM + categorical_crossentropy
{'loss': 0.32302114367485046, 'accuracy': 0.9103448390960693, 'precision': 0.9103448390960693, 'recall': 0.9103448390960693, 'auc': 0.9054222106933594}

### ADAM + Focal Loss
{'loss': 0.06994660943746567, 'accuracy': 0.9068965315818787, 'precision': 0.9068965315818787, 'recall': 0.9068965315818787, 'auc': 0.9103567004203796}


121528, 133546, 144443